In [ ]:
# Setup the Jupyter version of Dash
# JupyterDash allows Dash applications to run cleanly inside Jupyter notebooks
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl                    # Leaflet maps for geospatial visualization
from dash import dcc, html                   # Core Dash UI components
import plotly.express as px                  # High-level plotting library
from dash import dash_table                  # Interactive data tables
from dash.dependencies import Input, Output
import base64                                # Used to encode image assets

JupyterDash.infer_jupyter_proxy_config()     # Ensures compatibility with Jupyter environments

# Configure data and plotting utilities
import pandas as pd                          # Primary data structure for tabular data

# Import the CRUD abstraction layer
# This separates database logic from presentation logic
from CRUD_Python_Module import AnimalShelter


###########################
# Data Manipulation / Model
###########################

# Database connection configuration
# Centralizing these values improves maintainability and reuse
USERNAME = "aacuser"
PASSWORD = "CSharpSNHU"
HOST = "localhost"
PORT = 27017
DATABASE = "AAC"
COLLECTION = "animals"

# Initialize the database interface
# This object encapsulates all MongoDB interactions
db = AnimalShelter(USERNAME, PASSWORD, HOST, PORT, DATABASE, COLLECTION)

# Retrieve all records for initial load
df = pd.DataFrame.from_records(db.read({}))

# Remove MongoDB ObjectID to ensure Dash compatibility
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)


#########################
# Dashboard Layout / View
#########################

app = JupyterDash(__name__)

# Load and encode Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Define the UI layout using a declarative structure
app.layout = html.Div([

    # Branding element
    html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'width': '200px'}
    ),

    # Dashboard title and identification
    html.Center(html.B(html.H1('CS-340 Dashboard – Cameron'))),
    html.Hr(),

    # Interactive filtering controls
    html.Div([
        html.Label("Rescue Type Filter"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DIT'},
                {'label': 'Reset', 'value': 'RESET'}
            ],
            value='RESET',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        )
    ]),
    html.Hr(),

    # Interactive data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'padding': '5px'}
    ),

    html.Hr(),

    # Visualization area (chart + map)
    html.Div(style={'display': 'flex'}, children=[
        html.Div(id='graph-id', style={'width': '50%'}),
        html.Div(id='map-id', style={'width': '50%'})
    ])
])


#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_dashboard(filter_type):
    """
    Applies domain-specific filtering rules using MongoDB queries.
    Demonstrates algorithmic decision-making and database querying.
    """

    rescue_queries = {
        'WR': {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "age_upon_outcome_in_weeks": {"$lte": 156},
            "sex_upon_outcome": "Intact Male"
        },
        'MWR': {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "age_upon_outcome_in_weeks": {"$lte": 156},
            "sex_upon_outcome": "Intact Male"
        },
        'DIT': {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "age_upon_outcome_in_weeks": {"$lte": 156},
            "sex_upon_outcome": "Intact Male"
        }
    }

    query = rescue_queries.get(filter_type, {})
    df_filtered = pd.DataFrame.from_records(db.read(query))

    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)

    return df_filtered.to_dict('records')


@app.callback(
    Output('graph-id', 'children'),
    Input('datatable-id', 'derived_virtual_data')
)
def update_graphs(view_data):
    """
    Generates a visual summary of breed distribution.
    Transforms raw data into a meaningful visual insight.
    """
    if not view_data:
        return []

    dff = pd.DataFrame(view_data)
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return dcc.Graph(figure=fig)


@app.callback(
    Output('map-id', 'children'),
    Input('datatable-id', 'derived_virtual_data'),
    Input('datatable-id', 'derived_virtual_selected_rows')
)
def update_map(view_data, selected_rows):
    """
    Displays geospatial information for a selected animal.
    Demonstrates integration of tabular and spatial data.
    """
    if not view_data or not selected_rows:
        return []

    dff = pd.DataFrame(view_data)
    row = selected_rows[0]

    return dl.Map(
        center=[30.75, -97.48],
        zoom=10,
        style={'width': '100%', 'height': '500px'},
        children=[
            dl.TileLayer(),
            dl.Marker(
                position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                children=[
                    dl.Tooltip(dff.iloc[row, 4]),
                    dl.Popup(html.P(dff.iloc[row, 9]))
                ]
            )
        ]
    )


# Launch the application
app.run_server()


Found 10000 document(s) matching the query.
Dash app running on https://mustangdistant-politichobby-3000.codio.io/proxy/8050/
Found 10000 document(s) matching the query.
Found 46 document(s) matching the query.
Found 5 document(s) matching the query.
Found 2 document(s) matching the query.
Found 10000 document(s) matching the query.
Found 10000 document(s) matching the query.
Found 46 document(s) matching the query.
Found 5 document(s) matching the query.
Found 2 document(s) matching the query.
Found 10000 document(s) matching the query.
